In [ ]:
import numpy as np
from scipy.integrate import simpson
import matplotlib.pyplot as plt
from scipy.special import comb
from qiskit import QuantumCircuit, transpile
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2, Session
import time as systime
import os

In [ ]:
# Definition of batches, the dimension d of each subsystem A and B, the frequency w, the number of partitions p and shots

batches = 3
d = 16
w = 0.1
p = 480
nshots = 8192
q = 2 * int(np.ceil(np.log(d) / np.log(2)))
num_qubits = 2*q + 1

In [ ]:
# Create a list to store job IDs and corresponding time values
job_info_hadamd16_qiskit_aachen = {}
# Create an array of time values
t_values = np.linspace(0, np.pi / w, p)

In [ ]:
# Initialize the analytical number of non-null Walsh angles and the angles/indices arrays
num_angles = int(q**2/4 + q)
a_j = np.zeros(num_angles, dtype=int)
indices = np.zeros(num_angles, dtype=int)

# Pre-calculate constants for angle computations
result_h1 = -(8**(q/2))*(1+2**(q/2))/8
result_h2 = (4**q)/16

# Calculate angles and indices
for j1 in range(int(q/2)):
    base_index = j1 + (j1+1)*int(q/2)
    
    a_j[j1] = result_h1/(2**j1)
    a_j[base_index] = a_j[j1]
    
    indices[j1] = 2**j1
    indices[base_index] = 2**(j1 + int(q/2))

    for j2 in range(int(q/2)):
        offset_index = base_index + j2 + 1
        a_j[offset_index] = result_h2/(2**(j2+j1))
        indices[offset_index] = indices[base_index] + 2**j2

In [ ]:
# This cell is complementary to the previous cell. Here we apply for both the original n qubit system and its copy all the cnots and z-rotations 
# This will result in the desired evolved state.

def gates(QC,position,c,t):
    binary_str = format(indices[position], f'0{q}b')[::-1]    
    msb_index = binary_str.rfind('1')

    #Applies the cnots (to the left of the z-rotation gate) for current a_j
    for i in range(q):
        if binary_str[i] == '1':
            if i != msb_index:
                QC.cx(i + 1 + c*q, msb_index + 1 + c*q)
    
    # Applies rotation gate to the current qubit
    theta_j = 2*a_j[position]*w*t/(2**q)
    QC.rz(theta_j, msb_index + 1 + c*q)
    
    # Applies cnots in reverse order
    for i in reversed(range(q)):
        if binary_str[i] == '1':
            if i != msb_index:
                QC.cx(i + 1 + c*q, msb_index + 1 + c*q)

In [ ]:
# SO RODAR SE O CODIGO FOR FINALIZADO SEM TERMINAR TODOS JOBS. VAI CRIAR O NPY A PARTIR DO TXT

#import numpy as np
#import re

#log_path = "batch_times_log.txt"
#job_info = {}

#with open(log_path, "r") as f:
#    for line in f:
#        match = re.match(r"t = ([0-9.eE+-]+), batch = (\d+), job_id = (\w+)", line)
#        if match:
#            t = float(match.group(1))
#            batch = int(match.group(2))
#            job_id = match.group(3)
#            job_info[(t, batch)] = job_id

# Save the reconstructed dictionary
#np.save("job_info_hadamd16_qiskit_aachen.npy", job_info)
#print(f"✅ Rebuilt job_info_hadamd16_qiskit_aachen.npy with {len(job_info)} entries.")


In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2
from qiskit import QuantumCircuit, transpile
import time as systime
import os
import numpy as np

# === Configurações ===
service = QiskitRuntimeService()
backend_name = "ibm_aachen"
backend = service.backend(backend_name)
sampler = SamplerV2(mode=backend)
sampler.options.dynamical_decoupling.enable = True
sampler.options.dynamical_decoupling.sequence_type = "XY4"

job_info_path = "job_info_hadamd16_qiskit_aachen.npy"

# Carrega ou inicializa o dicionário
job_info_hadamd16_qiskit_aachen = (
    np.load(job_info_path, allow_pickle=True).item()
    if os.path.exists(job_info_path) else {}
)

# Containers
results = {}
job_ids = {}

# === Define which batch to submit now ===
# For now: you are still collecting batch 0
current_batch = 0

# Go over all t values and submit batch 0 only where missing
print(f"\n🚀 Submitting missing batch {current_batch} jobs...")

submitted = 0
skipped = 0

for t in t_values:
    if (t, current_batch) in job_info_hadamd16_qiskit_aachen:
        print(f"  ⚠️ Already submitted t = {t}, batch = {current_batch}, skipping...")
        skipped += 1
        continue

    # === Build circuit ===
    qc = QuantumCircuit(num_qubits, 1)
    for j in range(1, 2*q + 1):
        qc.h(j)
    for c in range(2):
        for position in range(num_angles):
            gates(qc, position, c, t)
    qc.h(0)
    for i in range(1, int(q / 2) + 1):
        qc.cswap(0, i, i + q)
    qc.h(0)
    qc.measure(0, 0)

    transpiled = transpile(qc, backend=backend, optimization_level=3)

    batch_start = systime.time()
    while True:
        try:
            job = sampler.run([transpiled], shots=nshots)
            jid = job.job_id()
            job_ids.setdefault(t, []).append(jid)
            job_info_hadamd16_qiskit_aachen[(t, current_batch)] = jid
            print(f"  ✅ Enviado t={t:.6f}, batch={current_batch}, job ID: {jid}")
            break
        except Exception as e:
            print(f"⚠️ Retry necessário (batch={current_batch}, t={t}): {e}")
            systime.sleep(30)

    wait_start = systime.time()
    while job.status() not in ["DONE", "CANCELLED", "ERROR"]:
        elapsed = systime.time() - wait_start
        print(f"    ⏳ Status: {job.status()}, aguardando... {elapsed:.1f}s ({elapsed / 60:.2f} min)")
        systime.sleep(10)

    batch_end = systime.time()
    duration = batch_end - batch_start

    with open("batch_times_log.txt", "a") as f:
        f.write(f"t = {t}, batch = {current_batch}, job_id = {jid}, duration = {duration:.2f} sec ({duration / 60:.2f} min)\n")

    if job.status() == "DONE":
        try:
            result = job.result()
            counts = result[0].join_data().get_counts()
            results.setdefault(t, []).append(counts)
            print(f"    ✅ Feito. Counts: {counts}")
        except Exception as e:
            print(f"    ❌ Falha ao obter resultado: {e}")
    else:
        print(f"    ❌ Job falhou com status: {job.status()}")

    submitted += 1

# Save final dictionary
np.save(job_info_path, job_info_hadamd16_qiskit_aachen)
print(f"\n✅ Submissão concluída: {submitted} enviados, {skipped} já estavam prontos.")

In [ ]:
# CELL TO PROCESS INCOMPLETE DATA (SÓ EXECUTAR QUANDO TIVER OS 3 BATCHES)
import numpy as np
from qiskit_ibm_runtime import QiskitRuntimeService
import matplotlib.pyplot as plt

# Load job ID dictionary
job_info = np.load("job_info_hadamd16_qiskit_aachen.npy", allow_pickle=True).item()
service = QiskitRuntimeService()

available_t_values = sorted(set(t for (t, _) in job_info.keys()))

Tr_val = []
Tr_val_errors = []
all_trace = {t: [] for t in available_t_values}

cont = 0
for (t, batch), job_id in job_info.items():
    try:
        job = service.job(job_id)
        status = job.status()

        if status != "DONE":
            print(f"⏳ Skipping job {job_id} — status is {status}")
            continue

        result = job.result()
        counts = result[0].join_data().get_counts()
        p0 = sum(v for k, v in counts.items() if k[::-1][0] == '0') / nshots
        Tr = 2 * p0 - 1
        all_trace[t].append(Tr)

    except Exception as e:
        print(f"⚠️ Failed job {job_id} for t = {t}, batch = {batch}: {e}")
    cont += 1
    print (cont)

# Per-t visualization and final trace if complete
for t in available_t_values:
    traces = all_trace[t]
    batches_collected = len(traces)

    print(f"\n📊 t = {t:.4f} has {batches_collected}/{batches} batches")

    if batches_collected == 0:
        print(f"❌ No successful batches for t = {t}")
        continue

    plt.errorbar([t]*batches_collected, traces, fmt='o', capsize=5, label=f't = {t:.2f}')
    plt.xlabel('t')
    plt.ylabel(r'$\gamma_A$')
    plt.title(f'Reduced purity per batch (so far) for $t = {t:.2f}$')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    if batches_collected == batches:
        mean_trace = np.mean(traces)
        std_err = np.std(traces, ddof=1) / np.sqrt(batches)
        Tr_val.append(mean_trace)
        Tr_val_errors.append(std_err)
        print(f"📈 Finalized: Mean = {mean_trace:.6f}, StdErr = {std_err:.6f}")
    else:
        print(f"⏳ Not enough batches for t = {t} — skipping mean/error calculation.")


In [ ]:
# === Extract γ_A(t) from batch 0 only (no averaging) === SÓ USAR QUANDO TIVER 1 BATCH AINDA
import numpy as np
from qiskit_ibm_runtime import QiskitRuntimeService

# Load saved job info
job_info = np.load("job_info_hadamd16_qiskit_aachen.npy", allow_pickle=True).item()
service = QiskitRuntimeService()

# Sort time values
available_t_values = sorted(set(t for (t, _) in job_info if (t, 0) in job_info))  # only if batch 0 exists

Tr_val_single = []
cont = 0

for t, job_id in job_info.items():
    try:
        job = service.job(job_id)
        result = job.result()
        counts = result[0].join_data().get_counts()
        p0 = sum(v for k, v in counts.items() if k[::-1][0] == '0') / nshots
        Tr = 2 * p0 - 1
        Tr_val_single.append(Tr)
    except Exception as e:
        print(f"❌ Failed to retrieve t = {t}: {e}")
    cont += 1
    print(cont)

Tr_val = np.array(Tr_val_single)

print("✅ γ_A(t) extracted for batch 0.")

t_val = np.array(available_t_values)

# Save in NPZ format (structured and compact)
np.savez("results_hadamd16_qiskit_aachen.npz", t=t_val, Tr=np.array(Tr_val))

# Salvar em CSV com cabeçalho
data_to_save = np.column_stack((t_val, Tr_val))
np.savetxt("results_hadamd16_qiskit_aachen.csv", data_to_save,
           delimiter=",", header="t,Tr", comments="")


In [ ]:
# Calculate the divisors of the integer n
def get_all_divisors(n):
    # Initialize a list to store all divisors
    divisors = set()
    
    # Loop to find divisors
    for i in range(1, int(n ** 0.5) + 1):
        if n % i == 0:
            # If i is a divisor, n // i is also a divisor
            divisors.add(i)
            divisors.add(n // i)
    
    # Return the divisors in ascending order
    return sorted(divisors)

# Functions for calculating analytical Fourier modes and reduced purity
def results(n):
    # Auxiliary function for Fourier mode calculations
    divisors = get_all_divisors(n)
    z = len(divisors) - 1
    R = sum(divisors)
    z_red = int(np.ceil(z/2))
    s = 0
    zt = z + 1
    for i in range(z_red, z + 1):
        if divisors[i] >= 2**(q/2):
            if divisors[i] != divisors[z - i]:
                s += divisors[i] + divisors[z - i]
                zt -= 2
            else:
                s += divisors[i]
                zt -= 1
    return R - s, zt

def calc_alpha_n_analyt(n):
    # Analytical calculation of Fourier modes
    Rt, Zt = results(n)
    if n <= (2**(q/2)-1)*(2**(q/2)-1):
        return 4 / (4**q) * (Zt * (2**q + n) - (2**(q/2 + 1)) * Rt)
    else:
        return 0

def Trace_analytical(t):
    # Analytical calculation of the trace/reduced purity
    Tr_part = sum(calc_alpha_n_analyt(v) * np.cos(v * w * t) for v in range(1, int((2**(q/2) - 1)**2 + 1)))
    Tr_analit = ((2 * 8**(q/2) - 2**q) / 4**q) + Tr_part
    return Tr_analit

In [ ]:
# CÉLULA QUE OTIMIZA O FATOR DE CORREÇÃO, INDEPENDENTE DA CÉLULA DE PLOT ORIGINAL
from scipy.optimize import minimize_scalar

# Carrega os dados do CSV
t_val, Tr_val_exp = np.loadtxt("results_hadamd16_qiskit_aachen.csv", 
                                          delimiter=",", 
                                          skiprows=1, 
                                          unpack=True)

# === Cópias seguras ===
n_val = np.arange(2, 2*(d-1)+1)

Tr_val_analyt = np.array(Trace_analytical(t_val))

# === Funções para αₙ e erro ===
def calc_alpha_n_exp(n, Tr_vals):
    integrando = (2*w/np.pi) * Tr_vals * np.cos(n * w * t_val)
    result = simpson(integrando, x=t_val)
    return result

# === Limite inferior analítico B_n
def analytical_bound(n):
    n = np.asarray(n)
    cond = (n >= 2) & (n < 2**(q/2))
    result = np.zeros_like(n, dtype=float)
    result[cond] = (8 / 2**(2*q)) * (-(2**(q/2) - 1)*n[cond] + 2**(q/2)*(2**(q/2) - 1))
    return result


# Region 2 : Below alpha_p2 in [2, 2(d-1)] → "possibly prime"
def alpha_p2(x):
    base = (4/(2**(q/2))**4)*((2**(q/2))**2 + x - 2 * (2**(q/2)) * np.sqrt(x))
    return np.where(x < d, base + analytical_bound(x), base)
alpha_p2 = alpha_p2(n_val)

# === αₙ corrigidos para um fator arbitrário (só valores)
def calc_alpha_scaled_exp(scale_factor, Tr_vals):
    scaled_vals = scale_factor * np.copy(Tr_vals)
    return np.array([calc_alpha_n_exp(n, scaled_vals) for n in n_val])

# === Função objetivo: soma das diferenças absolutas
def objective_exp(scale_factor, Tr_vals, alpha_analytical):
    alpha_scaled = calc_alpha_scaled_exp(scale_factor, Tr_vals)
    return np.sum(np.abs(alpha_scaled - alpha_analytical))

# === Modos analíticos
alpha_n_analyt = np.array([calc_alpha_n_analyt(n) for n in n_val])

# === Otimização
optimize_result_exp = minimize_scalar(
    objective_exp,
    bounds=(0.1, 3.0),
    method='bounded',
    args=(Tr_val_exp, alpha_n_analyt)
)

optimal_scale_factor_exp = optimize_result_exp.x
print(f"🔵 Melhor fator de escala experimental encontrado: {optimal_scale_factor_exp:.6f}")

# === Recalcular modos corrigidos (usando erro original)
Tr_val_exp_corrected = optimal_scale_factor_exp * Tr_val_exp
alpha_n_exp_corrected = np.array([calc_alpha_n_exp(n, Tr_val_exp_corrected) for n in n_val])

# === Modos originais e simulados
alpha_n_exp_original = np.array([calc_alpha_n_exp(n, Tr_val_exp) for n in n_val])
bound_n = [analytical_bound(n) for n in n_val]

# === Gráfico 1: pureza vs tempo
plt.figure()
plt.plot(t_val, Tr_val_analyt, label='Analytical', color='red')
plt.plot(t_val, Tr_val_exp, label='Demonstration (Original)', color='blue')
plt.plot(t_val, Tr_val_exp_corrected, label='Demonstration (Mitigated)', color='darkorange')
plt.xlabel('t', fontsize=16)
plt.ylabel(r'$\gamma_A(t)$', fontsize=16)
plt.title(f'Purity vs time for $d = {d}$', fontsize=18)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)
plt.legend(fontsize=9.5)
plt.tight_layout()
plt.savefig("purity_vs_time_d16.pdf", dpi=300, bbox_inches='tight')
plt.show()

# === Gráfico 2: modos de Fourier
plt.figure()
plt.plot(n_val, alpha_n_analyt, marker='o', linestyle='-', label='Analytical', color='red')
plt.plot(n_val, alpha_n_exp_original, marker='o', label='Demonstration (Original)', color='blue')
plt.plot(n_val, alpha_n_exp_corrected, marker='o',
             label='Demonstration (Mitigated)', color='darkorange')
plt.plot(n_val, bound_n, label=r'Lower bound $B_n$', color='green')
plt.plot(n_val, alpha_p2, linewidth=2.2,label=r'Composite bound $P_n$',color='purple')


end = int((d - 1)**2)
y_alpha_p2_full = alpha_p2[(n_val >= 2) & (n_val <= end)]
x_full = n_val[(n_val >= 2) & (n_val <= end)]
plt.fill_between(x_full, bound_n, y_alpha_p2_full, color='cornflowerblue', alpha=0.7, label='Possibly prime')
plt.xlabel('n', fontsize=16)
plt.ylabel(r'$\alpha_n$', fontsize=16)
plt.title(f'Fourier modes for $d = {d}$', fontsize=18)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.legend(fontsize=9.5)
plt.grid(True)
plt.tight_layout()
plt.savefig("fourier_modes_d16.pdf", dpi=300, bbox_inches='tight')
plt.show()